In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating new project at `c:\Users\nikla\git\AMOProject`
  No Changes to `C:\Users\nikla\git\AMOProject\Project.toml`
  No Changes to `C:\Users\nikla\git\AMOProject\Manifest.toml`


In [53]:
using JuMP, Distributions, HiGHS, Clustering, Distances, Plots, Statistics, StatsBase, Ipopt
gr()

Plots.GRBackend()

In [13]:
# Generate 288 distribution vectors
function generate_distributions(si::Float64, consumption_mean::Float64, temp_mean::Float64, hour::Int)
    if hour < 6 || hour > 22
        solar = Beta(1, 1000)  # Close to zero but keeps Beta distribution structure
        temperature = Normal(temp_mean-4, 9)
        consumption = Normal(consumption_mean/(consumption_mean-1), consumption_mean / 11)    
    else
        β = ((si * (1 + si) / (si / 10)) - 1) * (1 - si)
        α = si * β / (1 - si)
        solar = Beta(α, β)
        temperature = Normal(temp_mean, 9)
        consumption = Normal(consumption_mean, consumption_mean / 11)    

    end
    return [solar, consumption, temperature]
end

dist_vec = Vector{Vector{Distribution}}(undef, 12 * 24)

for month in 1:12
    for hour in 1:24
        # Assume some seasonal variations
        si = 0.25 + 0.1 * sin(2π * month / 12)  # Varying solar irradiance
        si = hour < 6 || hour > 22 ? 0.0 : si   # Zero irradiance at night
        consumption_mean = 90.0 + 10 * sin(2π * hour / 24)  # Daily load cycle
        temp_mean = 13.0 + 5 * cos(2π * month / 12)  # Seasonal temperature
        
        # Store the distribution vector
        dist_vec[(month - 1) * 24 + hour] = generate_distributions(si, consumption_mean, temp_mean, hour)
    end
end

# Check an example
dist_vec[1]  # First hour of January

3-element Vector{Distribution}:
 Beta{Float64}(α=1.0, β=1000.0)
 Normal{Float64}(μ=1.0109184382295962, σ=8.417108222820474)
 Normal{Float64}(μ=13.330127018922195, σ=9.0)

In [21]:
# Function to sample, cluster, and compute probabilities
function cluster_scenarios_with_probabilities(dist_vec, num_samples=5000, num_clusters=5)
    scenarios = []
    probabilities = []
    
    for (i, dist_set) in enumerate(dist_vec)
        # Generate 5000 samples for each variable
        samples = hcat(rand(dist_set[1], num_samples),  # Solar Irradiance
                       rand(dist_set[2], num_samples),  # Energy Consumption
                       rand(dist_set[3], num_samples)) # Temperature
        
        # Ensure non-negativity
        samples[:, 1] .= max.(samples[:, 1], 0)  # Solar Irradiance ≥ 0
        samples[:, 2] .= max.(samples[:, 2], 0)  # Consumption ≥ 0

        # Apply k-means clustering
        result = kmeans(samples', num_clusters)  # Clustering (transpose for correct shape)
        centroids = result.centers'  # Get cluster centers (transpose back)
        
        # Compute probabilities (relative frequency of points in each cluster)
        cluster_counts = countmap(result.assignments)  # Count points per cluster
        probs = [cluster_counts[j] / num_samples for j in 1:num_clusters]  # Normalize

        # Store results
        push!(scenarios, centroids)
        push!(probabilities, probs)
    end
    
    return scenarios, probabilities
end

# Generate 5 scenarios and their probabilities for all 288 time periods
scenarios_5, scenario_probs = cluster_scenarios_with_probabilities(dist_vec)

# Check an example
println("Scenarios for January, 1st hour: ", scenarios_5[1]) 
println("Probabilities for January, 1st hour: ", scenario_probs[1], " sum ", sum(scenario_probs[1]))  

Scenarios for January, 1st hour: [0.0010280930697278133 1.6523667043996533 0.9890543072374182; 0.0009951774445681 11.219511481003131 20.018763702778173; 0.0009478371166922513 1.5519123836840234 24.74745862624762; 0.0010057383473341588 1.3548433977594132 12.745069837658939; 0.0010046247756005603 12.212646737195456 6.921588650536541]
Probabilities for January, 1st hour: [0.193, 0.1212, 0.2004, 0.3686, 0.1168] sum 1.0


## Optimization Model

### Scalars

In [63]:
T = 288  # 12 months * 24 hours
num_scenarios = 5  # 5 clusters per time step

SCC = 50  # Social cost of carbon ($/ton CO2)
# E_cap = 1000 Emission cap
C_inv = 10 * 12  # Investment cost (per day for 12 days per PV module)
C_op = 0,1 * 12 # operation cost per day for 12 days

c_grid_buy = 0.25  # Cost of buying from grid ($/kWh)
c_grid_feed = 0.10  # Revenue from feeding into the grid ($/kWh)
e_grid = 0.5  # Emission intensity of grid (tCO2/kWh)

# PV Parameters
η_0 = 0.18  # Nominal PV efficiency at 25°C
β = 0.004  # Temperature coefficient (per °C)
T_ref = 25  # Reference temperature (°C)
A = 1  # Area of PV system (arbitrary unit, related to y_PV)

function efficiency(T_t)
    return η_0 * (1 - β * (T_t - T_ref))  # Efficiency function
end


# Solar irradiance: column 1
S_t_omega = [[scenarios_5[t][ω, 1] for ω in 1:num_scenarios] for t in 1:T]

# Consumption: column 2
L_t_omega = [[scenarios_5[t][ω, 2] for ω in 1:num_scenarios] for t in 1:T]

# Temperature: column 3
T_t_omega = [[scenarios_5[t][ω, 3] for ω in 1:num_scenarios] for t in 1:T]

# Scenario probabilities
Φ_t_omega = [[scenario_probs[t][ω] for ω in 1:num_scenarios] for t in 1:T] 

288-element Vector{Vector{Float64}}:
 [0.193, 0.1212, 0.2004, 0.3686, 0.1168]
 [0.125, 0.1094, 0.195, 0.3482, 0.2224]
 [0.1158, 0.2184, 0.3606, 0.1154, 0.1898]
 [0.1604, 0.1514, 0.1418, 0.2672, 0.2792]
 [0.2136, 0.1982, 0.1186, 0.3658, 0.1038]
 [0.2476, 0.1586, 0.2282, 0.209, 0.1566]
 [0.2524, 0.1816, 0.1972, 0.1862, 0.1826]
 [0.2518, 0.156, 0.2124, 0.2304, 0.1494]
 [0.262, 0.1566, 0.1532, 0.2274, 0.2008]
 [0.201, 0.2322, 0.2818, 0.134, 0.151]
 ⋮
 [0.1758, 0.1914, 0.1802, 0.1888, 0.2638]
 [0.1536, 0.1558, 0.2036, 0.2676, 0.2194]
 [0.2662, 0.1776, 0.132, 0.2104, 0.2138]
 [0.2158, 0.2886, 0.137, 0.217, 0.1416]
 [0.1816, 0.1646, 0.1482, 0.2436, 0.262]
 [0.1452, 0.1658, 0.208, 0.2568, 0.2242]
 [0.203, 0.1854, 0.2578, 0.1686, 0.1852]
 [0.111, 0.3578, 0.1886, 0.1148, 0.2278]
 [0.278, 0.1474, 0.2764, 0.1326, 0.1656]

### Model

In [64]:
model = Model(Ipopt.Optimizer)

# Upper Level variables
@variable(model, 0 <= y_PV <= 30)  # PV Capacity and upper bound

# Lower Level Variables (Operational Dispatch)
@variable(model, G_in[t=1:T, ω=1:num_scenarios] >= 0)  # Grid import
@variable(model, G_out[t=1:T, ω=1:num_scenarios] >= 0)  # Grid feed-in
@variable(model, P_PV[t=1:T, ω=1:num_scenarios] >= 0)  # PV production

# Meeting the demand
@constraint(model, [t=1:T, ω=1:num_scenarios],
    L_t_omega[t][ω] == y_PV * P_PV[t, ω] + G_in[t, ω] - G_out[t, ω]
)
# Lower Level Objective: Minimize Operating Cost
@expression(model, C_op,
    sum(Φ_t_omega[t][ω] * (c_grid_buy * G_in[t, ω] - c_grid_feed * G_out[t, ω]) 
        for t in 1:T, ω in 1:num_scenarios)
)

# PV production is calculated based on efficiency function
@constraint(model, [t=1:T, ω=1:num_scenarios],
    P_PV[t, ω] == efficiency(T_t_omega[t][ω]) * A * S_t_omega[t][ω]
)

# Emissions Calculation
@expression(model, E_op,
    sum(Φ_t_omega[t][ω] * e_grid * G_in[t, ω] for t in 1:T, ω in 1:num_scenarios)
)

# Upper Level Objective: Minimize Total Cost
@objective(model, Min,
    C_inv * y_PV + C_op # Investment Cost + Operating Cost
    + C_op  # Expected Operating Cost
    + SCC * E_op  # Expected Emissions Cost
)

120 y_PV + 4.9215 G_in[1,1] - 0.0386 G_out[1,1] + 3.0906000000000002 G_in[1,2] - 0.02424 G_out[1,2] + 5.1102 G_in[1,3] - 0.040080000000000005 G_out[1,3] + 9.3993 G_in[1,4] - 0.07372 G_out[1,4] + 2.9783999999999997 G_in[1,5] - 0.023360000000000002 G_out[1,5] + 3.1875 G_in[2,1] - 0.025 G_out[2,1] + 2.7897 G_in[2,2] - 0.02188 G_out[2,2] + 4.9725 G_in[2,3] - 0.03900000000000001 G_out[2,3] + 8.8791 G_in[2,4] - 0.06964000000000001 G_out[2,4] + 5.6712 G_in[2,5] - 0.04448 G_out[2,5] + 2.9529 G_in[3,1] - 0.02316 G_out[3,1] + 5.5692 G_in[3,2] - 0.043680000000000004 G_out[3,2] + 9.1953 G_in[3,3] - 0.07212 G_out[3,3] + 2.9427000000000003 G_in[3,4] - 0.023080000000000003 G_out[3,4] + 4.8399 G_in[3,5] - [[...2821 terms omitted...]] + 5.1765 G_in[286,1] - 0.040600000000000004 G_out[286,1] + 4.7277 G_in[286,2] - 0.03708 G_out[286,2] + 6.573899999999999 G_in[286,3] - 0.051559999999999995 G_out[286,3] + 4.2993 G_in[286,4] - 0.03372 G_out[286,4] + 4.7226 G_in[286,5] - 0.037040000000000003 G_out[286,5] + 

### Solving

In [65]:
optimize!(model)


This is Ipopt version 3.14.17, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:     7200
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     1440

Total number of variables............................:     4321
                     variables with only lower bounds:     4320
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:     2880
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  7.4063926e+01 1.13e+02 6.45e+01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [66]:
y_PV_value = value(y_PV)
println("Optimal PV Capacity (y_PV): $y_PV_value")

Optimal PV Capacity (y_PV): 30.000000299976083
